In [1]:
import gc
import os
import pickle
import random
from collections import defaultdict
from pathlib import Path
from time import time

import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import polars as pl
import seaborn as sns
from sklearn.metrics import (accuracy_score, average_precision_score,
                             cohen_kappa_score, explained_variance_score,
                             f1_score, log_loss, mean_absolute_error,
                             mean_squared_error, mean_squared_log_error,
                             median_absolute_error, precision_score, r2_score,
                             recall_score, roc_auc_score)
from sklearn.model_selection import GroupKFold, KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_sample_weight
from tqdm import tqdm

plt.style.use("ggplot")
tqdm.pandas()

In [2]:
class CFG:
    name = "exp005"
    seed = 42
    n_fold = 5

    path_input = Path("../input")
    path_output = Path("../output")

    candidate_name_list = [
        "past_view_yado",
        "top_popular_yado",
        "top_latest_next_booking",
    ]
    feature_name_list = [
        "top_latest_next_booking",
        "past_view_yado",
        "top_popular_yado",
        "top_wid_popular_yado",
        "top_ken_popular_yado",
        "top_lrg_popular_yado",
        "top_sml_popular_yado",
    ]

    cat_features = ["yad_no", "latest_yad_no", "wid_cd", "ken_cd", "lrg_cd", "sml_cd"]

    topn_top_popular_yado = 20
    topn_top_latest_next_booking = 20

    lgb_model_params = {
        "objective": "binary",
        "metric": "auc",
        "verbosity": -1,
        "learning_rate": 0.1,
        "random_state": seed,
    }

    lgb_train_params = {
        "num_boost_round": 999999,
    }

In [3]:
def set_seed(seed=42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    # torch.manual_seed(seed)
    # torch.cuda.manual_seed(seed)
    # torch.backends.cudnn.deterministic = True


def setup(CFG):
    # expフォルダを作成
    CFG.path_exp = CFG.path_output / CFG.name
    CFG.path_exp.mkdir(parents=True, exist_ok=True)

    # seedの設定
    set_seed(CFG.seed)

    return CFG

In [4]:
CFG = setup(CFG)

In [5]:
# データの読み込み
train_log_df = pl.read_csv(CFG.path_input / "train_log.csv")
train_label_df = pl.read_csv(CFG.path_input / "train_label.csv")

test_log_df = pl.read_csv(CFG.path_input / "test_log.csv")
test_session_df = pl.read_csv(CFG.path_input / "test_session.csv")

yado_df = pl.read_csv(CFG.path_input / "yado.csv")
yado_embedding_df = pl.read_parquet(CFG.path_input / "image_embeddings.parquet")

In [6]:
# すべてのログデータはあとあと参照をするので先に作っておきます.
whole_log_df = pl.concat([train_log_df, test_log_df])

In [7]:
flag_column = [
    "wireless_lan_flg",
    "onsen_flg",
    "kd_stn_5min",
    "kd_bch_5min",
    "kd_slp_5min",
    "kd_conv_walk_5min",
]
# 欠損を0で埋める
yado_df[flag_column] = yado_df[flag_column].fill_null(0)

# make candidate

In [8]:
# Group Kfold, labelにfold情報を付けておく。
kf = KFold(n_splits=CFG.n_fold, shuffle=True, random_state=CFG.seed)
fold_assignments = np.full(train_label_df.height, -1, dtype=int)  # heightは行数
for i, (_, valid_index) in enumerate(kf.split(train_label_df)):
    fold_assignments[valid_index] = i
# foldの情報を付与
train_label_df = train_label_df.with_columns(pl.Series("fold", fold_assignments))

In [9]:
def create_past_view_yado_candidates(log: pl.DataFrame):
    """
    過去に見た宿の候補を作成する。直近のものは除外することに注意
    """
    # session_idの最大値
    max_seq_no = log.group_by("session_id").agg(pl.max("seq_no").alias("max_seq_no"))
    log = log.join(max_seq_no, on="session_id")

    # 最大値に該当する行を除外
    past_yado_candidates = log.filter(pl.col("seq_no") != pl.col("max_seq_no"))
    past_yado_candidates = past_yado_candidates.select(
        ["session_id", "yad_no"]
    ).unique()

    # 簡易的な特徴量
    # 何個前に見たか
    past_yado_feature = log.with_columns(
        (pl.col("max_seq_no") - pl.col("seq_no")).alias("max_seq_no_diff")
    ).filter(pl.col("seq_no") != pl.col("max_seq_no"))
    # 同じやつを見ていたら、最大値を取る
    past_yado_feature = past_yado_feature.join(
        past_yado_feature.group_by(["session_id", "yad_no"]).agg(
            pl.col("max_seq_no_diff").max().alias("max_seq_no_diff")
        ),
        on=["session_id", "yad_no", "max_seq_no_diff"],
    )

    # 何回見たか
    session_view_count = (
        log.group_by(["session_id", "yad_no"])
        .count()
        .rename({"count": "session_view_count"})
    )
    past_yado_feature = past_yado_feature.join(
        session_view_count, how="left", on=["session_id", "yad_no"]
    ).drop("seq_no")

    return past_yado_candidates, past_yado_feature

In [10]:
def create_topN_popular_yado_candidates(train_label_df, train_test="train", top=10):
    """
    予約された人気宿を候補として作成。リークに注意
    """
    top_yado_candidate = pl.DataFrame()
    popular_yado_feature = pl.DataFrame()
    if train_test == "train":
        for fold in range(CFG.n_fold):
            # valでない部分を取得
            train_label = train_label_df.filter(pl.col("fold") != fold)
            # yad_noでグループ化して、予約数をカウント
            popular_yado_sort = (
                train_label["yad_no"].value_counts().sort(by="counts", descending=True)
            )

            # 候補作成
            top_yado_candidate_fold = (
                popular_yado_sort.head(top)
                .with_columns(pl.lit(fold).alias("fold"))
                .select(["yad_no", "fold"])
            )
            top_yado_candidate = pl.concat(
                [top_yado_candidate, top_yado_candidate_fold]
            )

            # 簡易的な特徴量
            popular_yado_feature_fold = popular_yado_sort.with_columns(
                pl.lit(fold).alias("fold")
            )
            # 予約回数の順位
            popular_yado_feature_fold = popular_yado_feature_fold.with_columns(
                pl.arange(1, len(popular_yado_feature_fold) + 1).alias("popular_rank")
            )
            popular_yado_feature = pl.concat(
                [popular_yado_feature, popular_yado_feature_fold]
            )
    else:  # testは全体で作成
        # yad_noでグループ化して、予約数をカウント
        popular_yado_sort = (
            train_label_df["yad_no"].value_counts().sort(by="counts", descending=True)
        )

        # 候補作成
        top_yado_candidate = popular_yado_sort.head(top).select(["yad_no"])
        # 予約回数の順位
        popular_yado_feature = popular_yado_sort.with_columns(
            pl.arange(1, len(popular_yado_sort) + 1).alias("popular_rank")
        )

    popular_yado_feature = popular_yado_feature.rename({"counts": "reservation_counts"})

    return top_yado_candidate, popular_yado_feature

In [11]:
def create_topN_area_popular_yado_candidates(
    train_label_df, yado_df, train_test="train", area="wid_cd", top=10
):
    """
    エリア単位での人気宿を候補として作成。リークに注意
    """
    # yad_noでマージ
    label_yado = train_label_df.join(yado_df, on="yad_no")
    top_yado_area_candidate = pl.DataFrame()
    popular_yado_area_feature = pl.DataFrame()
    if train_test == "train":
        for fold in range(CFG.n_fold):
            # valでない部分を取得
            train_label = label_yado.filter(pl.col("fold") != fold)
            # areaとyad_noでグループ化して、予約数をカウント
            popular_yado_sort = (
                train_label.group_by([area, "yad_no"])
                .count()
                .sort(by=[area, "count"], descending=[False, True])
            )

            # 候補作成
            top_yado_area_candidate_fold = (
                popular_yado_sort.group_by(area)
                .head(top)
                .with_columns(pl.lit(fold).alias("fold"))
                .select([area, "yad_no", "fold"])
            )
            top_yado_area_candidate = pl.concat(
                [top_yado_area_candidate, top_yado_area_candidate_fold]
            )

            # 簡易的な特徴量
            popular_yado_area_feature_fold = popular_yado_sort.with_columns(
                pl.lit(fold).alias("fold")
            )
            # 予約回数の順位
            popular_yado_area_feature_fold = popular_yado_area_feature_fold.group_by(
                area
            ).map_groups(
                lambda group: group.with_columns(
                    pl.col("count")
                    .rank(method="dense", descending=True)
                    .over(area)
                    .alias(f"popular_{area}_rank")
                )
            )
            popular_yado_area_feature = pl.concat(
                [popular_yado_area_feature, popular_yado_area_feature_fold]
            )

    else:  # testは全体で作成
        # areaとyad_noでグループ化して、予約数をカウント
        popular_yado_sort = (
            label_yado.group_by([area, "yad_no"])
            .count()
            .sort(by=[area, "count"], descending=[False, True])
        )

        # 候補作成
        top_yado_area_candidate = (
            popular_yado_sort.group_by(area).head(top).select([area, "yad_no"])
        )
        # 簡易的な特徴量
        popular_yado_area_feature = popular_yado_sort.group_by(area).map_groups(
            lambda group: group.with_columns(
                pl.col("count")
                .rank(method="dense", descending=True)
                .over(area)
                .alias(f"popular_{area}_rank")
            )
        )

    popular_yado_area_feature = popular_yado_area_feature.drop("count")

    return top_yado_area_candidate, popular_yado_area_feature

In [12]:
def create_latest_next_booking_topN_candidate(
    log_df, train_label_df, train_test="train", top=10
):
    """
    直近見た宿からどこが予約しやすいか
    """
    # 最後に見た宿
    log_latest = log_df.group_by("session_id").tail(1)
    # rename
    log_latest = log_latest.rename({"yad_no": "latest_yad_no"})
    log_latest = log_latest.join(train_label_df, on="session_id")

    # 学習データはtrain/validで分ける
    top_latest_next_booking_candidate = pl.DataFrame()
    top_latest_next_booking_feature = pl.DataFrame()
    if train_test == "train":
        for fold in range(CFG.n_fold):
            # 学習データのみ
            train_log_latest = log_latest.filter(pl.col("fold") != fold)
            # 最後に見た宿と予約した宿の情報を取得
            train_log_latest = (
                train_log_latest.group_by(["latest_yad_no", "yad_no"])
                .count()
                .sort(by=["latest_yad_no", "count"], descending=[False, True])
            )

            # 候補作成
            top_latest_next_booking_candidate_fold = (
                train_log_latest.group_by("latest_yad_no")
                .head(top)
                .with_columns(pl.lit(fold).alias("fold"))
                .select(["yad_no", "latest_yad_no", "fold"])
            )
            top_latest_next_booking_candidate = pl.concat(
                [
                    top_latest_next_booking_candidate,
                    top_latest_next_booking_candidate_fold,
                ]
            )

            # 簡易的な特徴量
            top_latest_next_booking_feature_fold = train_log_latest.with_columns(
                pl.lit(fold).alias("fold")
            )
            top_latest_next_booking_feature_fold = (
                top_latest_next_booking_feature_fold.group_by(
                    "latest_yad_no"
                ).map_groups(
                    lambda group: group.with_columns(
                        pl.col("count")
                        .rank(method="dense", descending=True)
                        .over("latest_yad_no")
                        .alias(f"latest_next_booking_rank")
                    )
                )
            )
            top_latest_next_booking_feature = pl.concat(
                [top_latest_next_booking_feature, top_latest_next_booking_feature_fold]
            )

    else:  # testは全体で作成
        # 最後に見た宿と予約した宿の情報を取得
        log_latest = (
            log_latest.group_by(["latest_yad_no", "yad_no"])
            .count()
            .sort(by=["latest_yad_no", "count"], descending=[False, True])
        )

        # 候補作成
        top_latest_next_booking_candidate = (
            log_latest.group_by("latest_yad_no")
            .head(top)
            .select(["yad_no", "latest_yad_no"])
        )
        # 簡易的な特徴量
        top_latest_next_booking_feature = log_latest.group_by(
            "latest_yad_no"
        ).map_groups(
            lambda group: group.with_columns(
                pl.col("count")
                .rank(method="dense", descending=True)
                .over("latest_yad_no")
                .alias(f"latest_next_booking_rank")
            )
        )

    top_latest_next_booking_feature = top_latest_next_booking_feature.drop("count")

    return top_latest_next_booking_candidate, top_latest_next_booking_feature

In [13]:
# 過去に見た宿の候補を作成
(
    train_past_view_yado_candidates,
    train_past_view_yado_feature,
) = create_past_view_yado_candidates(train_log_df)
(
    test_past_view_yado_candidates,
    test_past_view_yado_feature,
) = create_past_view_yado_candidates(test_log_df)

In [14]:
# 人気宿を候補として作成
(
    train_top_popular_yado_candidates,
    train_top_popular_yado_feature,
) = create_topN_popular_yado_candidates(
    train_label_df, train_test="train", top=CFG.topn_top_popular_yado
)
(
    test_top_popular_yado_candidates,
    test_top_popular_yado_feature,
) = create_topN_popular_yado_candidates(
    train_label_df, train_test="test", top=CFG.topn_top_popular_yado
)

In [15]:
# エリア単位での人気宿を候補として作成
(
    train_top_wid_popular_yado_candidates,
    train_top_wid_popular_yado_feature,
) = create_topN_area_popular_yado_candidates(
    train_label_df, yado_df, train_test="train", area="wid_cd", top=10
)
(
    test_top_wid_popular_yado_candidates,
    test_top_wid_popular_yado_feature,
) = create_topN_area_popular_yado_candidates(
    train_label_df, yado_df, train_test="test", area="wid_cd", top=10
)

# 県単位での人気宿を候補として作成
(
    train_top_ken_popular_yado_candidates,
    train_top_ken_popular_yado_feature,
) = create_topN_area_popular_yado_candidates(
    train_label_df, yado_df, train_test="train", area="ken_cd", top=10
)
(
    test_top_ken_popular_yado_candidates,
    test_top_ken_popular_yado_feature,
) = create_topN_area_popular_yado_candidates(
    train_label_df, yado_df, train_test="test", area="ken_cd", top=10
)

# 大エリア単位での人気宿を候補として作成
(
    train_top_lrg_popular_yado_candidates,
    train_top_lrg_popular_yado_feature,
) = create_topN_area_popular_yado_candidates(
    train_label_df, yado_df, train_test="train", area="lrg_cd", top=10
)
(
    test_top_lrg_popular_yado_candidates,
    test_top_lrg_popular_yado_feature,
) = create_topN_area_popular_yado_candidates(
    train_label_df, yado_df, train_test="test", area="lrg_cd", top=10
)

# 小エリア単位での人気宿を候補として作成
(
    train_top_sml_popular_yado_candidates,
    train_top_sml_popular_yado_feature,
) = create_topN_area_popular_yado_candidates(
    train_label_df, yado_df, train_test="train", area="sml_cd", top=10
)
(
    test_top_sml_popular_yado_candidates,
    test_top_sml_popular_yado_feature,
) = create_topN_area_popular_yado_candidates(
    train_label_df, yado_df, train_test="test", area="sml_cd", top=10
)

In [16]:
# 直近見た宿からどこが予約しやすいかを候補として作成
(
    train_top_latest_next_booking_candidates,
    train_top_latest_next_booking_feature,
) = create_latest_next_booking_topN_candidate(
    train_log_df,
    train_label_df,
    train_test="train",
    top=CFG.topn_top_latest_next_booking,
)
(
    test_top_latest_next_booking_candidates,
    test_top_latest_next_booking_feature,
) = create_latest_next_booking_topN_candidate(
    train_log_df,
    train_label_df,
    train_test="test",
    top=CFG.topn_top_latest_next_booking,
)

In [17]:
# parquet形式で保存
train_past_view_yado_candidates.write_parquet(
    CFG.path_exp / "train_past_view_yado_candidates.parquet"
)
train_past_view_yado_feature.write_parquet(
    CFG.path_exp / "train_past_view_yado_feature.parquet"
)
test_past_view_yado_candidates.write_parquet(
    CFG.path_exp / "test_past_view_yado_candidates.parquet"
)
test_past_view_yado_feature.write_parquet(
    CFG.path_exp / "test_past_view_yado_feature.parquet"
)

train_top_popular_yado_candidates.write_parquet(
    CFG.path_exp / "train_top_popular_yado_candidates.parquet"
)
train_top_popular_yado_feature.write_parquet(
    CFG.path_exp / "train_top_popular_yado_feature.parquet"
)
test_top_popular_yado_candidates.write_parquet(
    CFG.path_exp / "test_top_popular_yado_candidates.parquet"
)
test_top_popular_yado_feature.write_parquet(
    CFG.path_exp / "test_top_popular_yado_feature.parquet"
)

train_top_wid_popular_yado_candidates.write_parquet(
    CFG.path_exp / "train_top_wid_popular_yado_candidates.parquet"
)
train_top_wid_popular_yado_feature.write_parquet(
    CFG.path_exp / "train_top_wid_popular_yado_feature.parquet"
)
test_top_wid_popular_yado_candidates.write_parquet(
    CFG.path_exp / "test_top_wid_popular_yado_candidates.parquet"
)
test_top_wid_popular_yado_feature.write_parquet(
    CFG.path_exp / "test_top_wid_popular_yado_feature.parquet"
)

train_top_ken_popular_yado_candidates.write_parquet(
    CFG.path_exp / "train_top_ken_popular_yado_candidates.parquet"
)
train_top_ken_popular_yado_feature.write_parquet(
    CFG.path_exp / "train_top_ken_popular_yado_feature.parquet"
)
test_top_ken_popular_yado_candidates.write_parquet(
    CFG.path_exp / "test_top_ken_popular_yado_candidates.parquet"
)
test_top_ken_popular_yado_feature.write_parquet(
    CFG.path_exp / "test_top_ken_popular_yado_feature.parquet"
)

train_top_lrg_popular_yado_candidates.write_parquet(
    CFG.path_exp / "train_top_lrg_popular_yado_candidates.parquet"
)
train_top_lrg_popular_yado_feature.write_parquet(
    CFG.path_exp / "train_top_lrg_popular_yado_feature.parquet"
)
test_top_lrg_popular_yado_candidates.write_parquet(
    CFG.path_exp / "test_top_lrg_popular_yado_candidates.parquet"
)
test_top_lrg_popular_yado_feature.write_parquet(
    CFG.path_exp / "test_top_lrg_popular_yado_feature.parquet"
)

train_top_sml_popular_yado_candidates.write_parquet(
    CFG.path_exp / "train_top_sml_popular_yado_candidates.parquet"
)
train_top_sml_popular_yado_feature.write_parquet(
    CFG.path_exp / "train_top_sml_popular_yado_feature.parquet"
)
test_top_sml_popular_yado_candidates.write_parquet(
    CFG.path_exp / "test_top_sml_popular_yado_candidates.parquet"
)
test_top_sml_popular_yado_feature.write_parquet(
    CFG.path_exp / "test_top_sml_popular_yado_feature.parquet"
)

train_top_latest_next_booking_candidates.write_parquet(
    CFG.path_exp / "train_top_latest_next_booking_candidates.parquet"
)
train_top_latest_next_booking_feature.write_parquet(
    CFG.path_exp / "train_top_latest_next_booking_feature.parquet"
)
test_top_latest_next_booking_candidates.write_parquet(
    CFG.path_exp / "test_top_latest_next_booking_candidates.parquet"
)
test_top_latest_next_booking_feature.write_parquet(
    CFG.path_exp / "test_top_latest_next_booking_feature.parquet"
)

In [18]:
def get_session_id_list(log_df):
    return log_df.group_by("session_id").head(1).select(["session_id"])  # セッションidを取得

In [19]:
train_session_id = get_session_id_list(train_log_df)
train_session_id = train_session_id.join(
    train_label_df.select(["fold", "session_id"]), how="left", on="session_id"
)

test_session_id = get_session_id_list(test_log_df)

In [20]:
candidate_list = {}
candidate_list["train"] = []
candidate_list["test"] = []

for train_test in ["train", "test"]:
    for candidate_name in tqdm(CFG.candidate_name_list):
        candidate = pl.read_parquet(
            CFG.path_exp / f"{train_test}_{candidate_name}_candidates.parquet"
        )
        # セッションに関連する候補を追加（今の所はpast_view_yadoだけ？）
        if "session_id" in candidate.columns:
            candidate_list[train_test].append(
                candidate.select(["session_id", "yad_no"])
            )
        # 直近のやつ
        elif "latest_yad_no" in candidate.columns:
            if train_test == "train":
                # 最後に見た宿番号をセッションごとに取得
                latest_yad_no = (
                    train_log_df.group_by("session_id")
                    .tail(1)
                    .select(["session_id", "yad_no"])
                    .rename({"yad_no": "latest_yad_no"})
                )
                latest_yad_no = latest_yad_no.join(
                    train_label_df.select(["session_id", "fold"]),
                    how="left",
                    on="session_id",
                )
                latest_yad_no = latest_yad_no.with_columns(
                    pl.col("fold").cast(pl.Int32)
                )
                candidate = latest_yad_no.join(
                    candidate, how="inner", on=["latest_yad_no", "fold"]
                )
            else:
                latest_yad_no = (
                    test_log_df.group_by("session_id")
                    .tail(1)
                    .select(["session_id", "yad_no"])
                    .rename({"yad_no": "latest_yad_no"})
                )
                candidate = latest_yad_no.join(
                    candidate, how="inner", on=["latest_yad_no"]
                )
            candidate_list[train_test].append(
                candidate.select(["session_id", "yad_no"])
            )

        # それ以外（人気宿系はここに入る。target見てるから全てfoldがカラムに入ってるはず）
        else:
            if train_test == "train":
                if "fold" in candidate.columns:
                    candidate_all = pl.DataFrame()
                    for fold in range(CFG.n_fold):
                        candidate_fold = train_session_id.filter(
                            pl.col("fold") == fold
                        ).join(
                            candidate.filter(pl.col("fold") == fold).select(["yad_no"]),
                            how="cross",
                        )
                        candidate_all = pl.concat([candidate_all, candidate_fold])
            else:
                candidate_all = test_session_id.join(
                    candidate.select(["yad_no"]), how="cross"
                )
            candidate_list[train_test].append(
                candidate_all.select(["session_id", "yad_no"])
            )

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 29.08it/s]


In [21]:
train_candidate = pl.concat(candidate_list["train"]).unique()
test_candidate = pl.concat(candidate_list["test"]).unique()

In [22]:
del candidate_list
gc.collect()

0

# Make feature

In [23]:
# 実際に予約した宿を結合
train_candidate = train_candidate.join(
    train_label_df.rename({"yad_no": "target"}), how="left", on="session_id"
)
train_candidate = train_candidate.with_columns(pl.col("fold").cast(pl.Int32))

# targetの作成
train_candidate = train_candidate.with_columns(
    (pl.col("yad_no") == pl.col("target")).alias("target").cast(pl.Int8)
)

In [24]:
train_latest_yad_no = (
    train_log_df.group_by("session_id")
    .tail(1)
    .select(["session_id", "yad_no"])
    .rename({"yad_no": "latest_yad_no"})
)
test_latest_yad_no = (
    test_log_df.group_by("session_id")
    .tail(1)
    .select(["session_id", "yad_no"])
    .rename({"yad_no": "latest_yad_no"})
)

In [25]:
# 直近に見た宿を結合
train_candidate = train_candidate.join(train_latest_yad_no, how="left", on="session_id")
test_candidate = test_candidate.join(test_latest_yad_no, how="left", on="session_id")

In [26]:
CFG.feature_name_list

['top_latest_next_booking',
 'past_view_yado',
 'top_popular_yado',
 'top_wid_popular_yado',
 'top_ken_popular_yado',
 'top_lrg_popular_yado',
 'top_sml_popular_yado']

In [27]:
for train_test in ["train", "test"]:
    for feature_name in tqdm(CFG.feature_name_list):
        feature = pl.read_parquet(
            CFG.path_exp / f"{train_test}_{feature_name}_feature.parquet"
        )
        if train_test == "train":
            # for fold in range(CFG.fold_num):
            if "session_id" in feature.columns:
                train_candidate = train_candidate.join(
                    feature, how="left", on=["session_id", "yad_no"]
                )
            elif "latest_yad_no" in feature.columns:
                train_candidate = train_candidate.join(
                    feature, how="left", on=["fold", "latest_yad_no", "yad_no"]
                )
            else:
                train_candidate = train_candidate.join(
                    feature, how="left", on=["fold", "yad_no"]
                )
        else:
            if "session_id" in feature.columns:
                test_candidate = test_candidate.join(
                    feature, how="left", on=["session_id", "yad_no"]
                )
            elif "latest_yad_no" in feature.columns:
                test_candidate = test_candidate.join(
                    feature, how="left", on=["latest_yad_no", "yad_no"]
                )
            else:
                test_candidate = test_candidate.join(feature, how="left", on=["yad_no"])

  0%|          | 0/7 [00:00<?, ?it/s]

100%|██████████| 7/7 [00:00<00:00, 11.54it/s]


In [28]:
# 欠損を0で埋める
train_candidate = train_candidate.fill_null(0)
test_candidate = test_candidate.fill_null(0)

In [29]:
train_candidate = train_candidate.join(
    yado_df.select(
        [
            "yad_no",
            "yad_type",
            "total_room_cnt",
            "wireless_lan_flg",
            "onsen_flg",
            "kd_stn_5min",
            "kd_bch_5min",
            "kd_slp_5min",
            "kd_conv_walk_5min",
        ]
    ),
    how="left",
    on="yad_no",
)
test_candidate = test_candidate.join(
    yado_df.select(
        [
            "yad_no",
            "yad_type",
            "total_room_cnt",
            "wireless_lan_flg",
            "onsen_flg",
            "kd_stn_5min",
            "kd_bch_5min",
            "kd_slp_5min",
            "kd_conv_walk_5min",
        ]
    ),
    how="left",
    on="yad_no",
)

In [30]:
def create_num_picture_df(yado_df):
    # yad_noとcategoryごとのデータ件数
    _df = yado_df.group_by(["yad_no", "category"]).count()

    # ピボットテーブルに変換
    num_picture_df = _df.pivot("count", "yad_no", "category", "sum").sort("yad_no")

    # 欠損を0で埋める
    num_picture_df = num_picture_df.fill_null(0)

    return num_picture_df

In [31]:
num_picture_df = create_num_picture_df(yado_embedding_df)
num_picture_df

yad_no,room,exterior,facility,food,others
i64,u32,u32,u32,u32,u32
1,3,3,3,3,2
2,1,3,3,3,2
3,3,3,1,3,2
4,1,3,3,3,2
5,3,3,3,3,3
6,2,3,3,3,2
7,3,1,3,0,3
8,3,3,3,1,3
9,3,3,3,3,1


In [32]:
train_candidate = train_candidate.join(num_picture_df, how="left", on="yad_no")
test_candidate = test_candidate.join(num_picture_df, how="left", on="yad_no")

In [33]:
train_candidate

session_id,yad_no,target,fold,latest_yad_no,latest_next_booking_rank,max_seq_no,max_seq_no_diff,session_view_count,reservation_counts,popular_rank,wid_cd,popular_wid_cd_rank,ken_cd,popular_ken_cd_rank,lrg_cd,popular_lrg_cd_rank,sml_cd,popular_sml_cd_rank,yad_type,total_room_cnt,wireless_lan_flg,onsen_flg,kd_stn_5min,kd_bch_5min,kd_slp_5min,kd_conv_walk_5min,room,exterior,facility,food,others
str,i64,i8,i32,i64,u32,i64,i64,u32,u32,i64,str,u32,str,u32,str,u32,str,u32,i64,f64,f64,f64,f64,f64,f64,f64,u32,u32,u32,u32,u32
"""00063c614aca49…",5583,1,2,6001,2,1,1,1,22,3575,"""dc414a17890cfc…",85,"""223938a74a6099…",26,"""63083678169ddd…",17,"""33bfe292401fc7…",15,0,55.0,0.0,1.0,0.0,0.0,0.0,0.0,3,3,3,2,3
"""00177ad4bf0130…",9222,1,0,1823,2,2,1,1,35,2058,"""c312e07b7a5d45…",53,"""a1b872e7729b0f…",12,"""4f5e37e16dcf9c…",9,"""c36ee05a7a2258…",9,0,89.0,1.0,0.0,0.0,0.0,0.0,1.0,3,3,3,3,3
"""0019914aa5f1a7…",1013,1,0,1259,3,1,1,1,43,1386,"""46e33861f921c3…",106,"""c86352f5b57e80…",56,"""30e12d5f9c1a9e…",7,"""96dca36640fbfb…",7,0,278.0,1.0,0.0,1.0,0.0,0.0,1.0,3,3,2,3,1
"""00781abf9ee127…",6766,1,1,7399,3,1,1,1,109,144,"""46e33861f921c3…",42,"""107c7305a74c8d…",22,"""52ca3d2824fc3c…",2,"""1cdef7807dc6bc…",2,0,89.0,1.0,0.0,1.0,0.0,0.0,1.0,3,3,3,3,3
"""007ea3ae39aef3…",1275,1,4,8163,3,1,1,1,88,258,"""f0112abf369fb0…",39,"""072c85e1653e10…",31,"""449c52ef581d5f…",17,"""f76e14a3f2ebd4…",12,0,653.0,1.0,0.0,1.0,0.0,0.0,1.0,3,3,3,3,2
"""00829452e8e21f…",5245,0,1,5245,0,2,2,2,43,1394,"""b07b75d367ebec…",59,"""0a66f6ab9c0507…",59,"""4713062d683b3b…",6,"""23a40c053a6e6b…",6,0,54.0,1.0,0.0,0.0,0.0,0.0,0.0,3,3,3,3,2
"""008aaae5680075…",6134,1,1,10750,0,1,1,1,32,2270,"""dc414a17890cfc…",69,"""223938a74a6099…",19,"""63083678169ddd…",10,"""33bfe292401fc7…",10,0,108.0,1.0,0.0,0.0,0.0,0.0,1.0,3,3,3,2,2
"""00c98f093bf74e…",224,0,2,4715,3,1,1,1,34,2083,"""46e33861f921c3…",116,"""83522288daa2f3…",13,"""0ded95208aa971…",11,"""0163d7c60f870a…",9,0,134.0,0.0,0.0,1.0,0.0,0.0,1.0,3,2,3,0,3
"""00e8a84d5dd001…",4201,1,4,235,2,1,1,1,58,767,"""e9316013ee1b03…",58,"""21a8fca4573868…",10,"""0a2df87481535e…",1,"""fa672d3370e4e6…",1,0,77.0,0.0,1.0,0.0,0.0,0.0,1.0,3,2,3,3,1


In [34]:
use_session_ids = (
    train_candidate.group_by("session_id")
    .agg(pl.col("target").sum())
    .filter(pl.col("target") == 1)["session_id"]
)

# Train model

In [35]:
lgb_model_list = []
pred = np.zeros(len(test_candidate))
for fold in range(CFG.n_fold):
    X_train = train_candidate.filter(
        (pl.col("session_id").is_in(use_session_ids)) & (pl.col("fold") != fold)
    ).drop(["fold", "target", "session_id"])
    Y_train = train_candidate.filter(
        (pl.col("session_id").is_in(use_session_ids)) & (pl.col("fold") != fold)
    )["target"].to_numpy()
    X_valid = train_candidate.filter(
        (pl.col("session_id").is_in(use_session_ids)) & (pl.col("fold") == fold)
    ).drop(["fold", "target", "session_id"])
    Y_valid = train_candidate.filter(
        (pl.col("session_id").is_in(use_session_ids)) & (pl.col("fold") == fold)
    )["target"].to_numpy()

    # pandasの方が扱いやすいので変換
    X_train = X_train.to_pandas()
    X_valid = X_valid.to_pandas()
    for feature in CFG.cat_features:
        X_train[feature] = X_train[feature].astype("category")
        X_valid[feature] = X_valid[feature].astype("category")

    train_dataset = lgb.Dataset(X_train, Y_train)
    valid_dataset = lgb.Dataset(X_valid, Y_valid)
    lgb_model = lgb.train(
        params=CFG.lgb_model_params,
        train_set=train_dataset,
        valid_sets=[train_dataset, valid_dataset],
        callbacks=[lgb.early_stopping(100), lgb.log_evaluation(1000)],
        **CFG.lgb_train_params,
    )

    lgb_model_list.append(lgb_model)

    del X_train, Y_train, X_valid, Y_valid, train_dataset, valid_dataset
    gc.collect()

Training until validation scores don't improve for 100 rounds
[1000]	training's auc: 0.983274	valid_1's auc: 0.943548
[2000]	training's auc: 0.987487	valid_1's auc: 0.945573
[3000]	training's auc: 0.989158	valid_1's auc: 0.946128
Early stopping, best iteration is:
[3316]	training's auc: 0.989509	valid_1's auc: 0.946284
Training until validation scores don't improve for 100 rounds
[1000]	training's auc: 0.983202	valid_1's auc: 0.943707
[2000]	training's auc: 0.987429	valid_1's auc: 0.945476
[3000]	training's auc: 0.989146	valid_1's auc: 0.946243
Early stopping, best iteration is:
[3833]	training's auc: 0.989961	valid_1's auc: 0.946458
Training until validation scores don't improve for 100 rounds
[1000]	training's auc: 0.983153	valid_1's auc: 0.944385
[2000]	training's auc: 0.987458	valid_1's auc: 0.946421
Early stopping, best iteration is:
[2886]	training's auc: 0.989014	valid_1's auc: 0.947084
Training until validation scores don't improve for 100 rounds
[1000]	training's auc: 0.982973

In [36]:
def apk(actual, predicted, k=10):
    """
    Computes the average precision at k for a single actual value.

    Parameters:
    actual : int
        The actual value that is to be predicted
    predicted : list
        A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements

    Returns:
    float
        The average precision at k
    """
    if actual in predicted[:k]:
        return 1.0 / (predicted[:k].index(actual) + 1)
    return 0.0


def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k for lists of actual values and predicted values.

    Parameters:
    actual : list
        A list of actual values that are to be predicted
    predicted : list
        A list of lists of predicted elements (order does matter in the lists)
    k : int, optional
        The maximum number of predicted elements

    Returns:
    float
        The mean average precision at k
    """
    return sum(apk(a, p, k) for a, p in zip(actual, predicted)) / len(actual)

In [37]:
def create_top_10_yad_predict(_df):
    # セッションごとに予測確率の高い順に yad_no の配列を作成
    _agg = (
        _df.sort_values("predict", ascending=False)
        .groupby("session_id")["yad_no"]
        .apply(list)
    )

    out_df = pd.DataFrame(index=_agg.index, data=_agg.values.tolist()).iloc[:, :10]

    return out_df

In [38]:
oof = pd.DataFrame()
test = test_candidate.with_columns(pl.lit(0).alias("predict"))
X_test = test.drop(["session_id"])
X_test = X_test.to_pandas()
for feature in CFG.cat_features:
    X_test[feature] = X_test[feature].astype("category")

for fold in range(CFG.n_fold):
    X_valid = train_candidate.filter((pl.col("fold") == fold)).drop(
        ["fold", "target", "session_id"]
    )
    Y_valid = train_candidate.filter((pl.col("fold") == fold))["target"].to_numpy()

    # pandasの方が扱いやすいので変換
    X_valid = X_valid.to_pandas()
    for feature in CFG.cat_features:
        X_valid[feature] = X_valid[feature].astype("category")

    X_valid["predict"] = lgb_model_list[fold].predict(X_valid)
    X_valid["session_id"] = train_candidate.filter((pl.col("fold") == fold))[
        "session_id"
    ].to_numpy()
    X_test["predict"] += (
        lgb_model_list[fold].predict(X_test.drop("predict", axis=1)) / CFG.n_fold
    )
    oof = pd.concat([oof, X_valid[["session_id", "predict", "yad_no"]]])
X_test["session_id"] = test["session_id"].to_numpy()

In [39]:
oof = oof.sort_values(["session_id", "predict"], ascending=False)
oof_ = create_top_10_yad_predict(oof)

In [40]:
oof_.to_csv(CFG.path_exp / "oof.csv")

In [41]:
pd_train_label_df = train_label_df.to_pandas()

In [42]:
mapk(
    actual=pd_train_label_df[
        pd_train_label_df["session_id"].isin(oof_.reset_index()["session_id"])
    ]
    .sort_values("session_id", ascending=True)["yad_no"]
    .to_list(),
    predicted=oof_.values.tolist(),
    k=10,
)

0.35779682360437065

In [43]:
sub = create_top_10_yad_predict(X_test)
sub.columns = [f"predict_{c}" for c in sub.columns]
sub = sub.reset_index(drop=True)

In [44]:
sub.to_csv(CFG.path_exp / "submission.csv", index=False)